# Abstract
#### Setting

        1. 1차 정제 데이터인 CAI_data_001.csv  데이터를 사용하여 2차 전처리 진행
        2. >를 기준으로 depth 나눔
        3. 모델링

#### Method
        
        1. VOC 유형 중 depth_1 유형을 예측하는 모델
        2. model 폴더에 pt파일로 모델 저장
        3. 현재 모델(KoBERT)와 타모델을 비교해봄
        4. BERTDataset class를 이용하여 데이터프레임을 입력하여 모델에 입력될 수 있도록 정제시킴

#### Result
        
        Val loss: 함께 출력되도록 커스터마이징 필요
        Val acc:  0.8895941840277778

#### Add
        1. epoch 늘리기 
        2. batch size 조정

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

from sklearn.model_selection import train_test_split

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
                                 

In [4]:
#GPU 사용
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

In [5]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [6]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df = pd.read_csv('/content/drive/MyDrive/AI/dataset/채널AI/CAI_data_001.csv')
df

,Label,sentence
0,원인불명>조치 전 자연 회복,고객 요청사 항 안됨상담 중 자연회복됨
1,실시간 채널>네트워크 오류,고객 요청사항 수신불인터넷 연결오류 뜸재부팅해 봄 모텔 영업용 일 중 꼭 점검원하여...
2,실시간 채널>수신 불량,고객 요청사항 수신불회선 단말특이사항 진단 결과 기타 추가 연락처 연후방
3,실시간 채널>네트워크 오류,수신 불 네트워크 오류 접수 보류
4,실시간 채널>수신 불량,고객 요청사항 수신불 연락 후 회선단말특이사항 진단 결과 기타 추가 연락처인 입
...,...,...
8181,실시간 채널>화질 이상,안심권유 고객 요청사항 화질 불량 시청 중 화면 멈추고 꺼졌다 켜지는 현상이 있다고...
8182,원인불명>조치 전 자연 회복,리콜 넷플릭스 연령 인증 문의 건 리콜하니 해결했다고 하심
8183,실시간 채널>수신 불량,고객 요청사항 대 수신불전송 후 리셋
8184,실시간 채널>네트워크 오류,안심권 유 빠른 점검 요청 고객 요청사항 시청 불인터넷 연결 오류 나 옴 모든 장비...


In [8]:
df['depth_1'] = df.Label.str.split('>').str[0]
df['depth_2'] = df.Label.str.split('>').str[1]
df['depth_3'] = df.Label.str.split('>').str[2]

In [9]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
result = le.fit_transform(df['depth_1'])
print(result)
print(le.classes_)

df['depth_1'] = result

[11  9  9 ...  9  9  3]
['AS접수/변경' 'VOD/컨텐츠' '고객원인' '단말' '데이터서비스' '리콜/손해배상' '부가/지능망서비스' '시설관리'
 '시설이전' '실시간 채널' '양방향서비스' '원인불명' '장애안내' '지니뮤직']


In [10]:
df.head()

,Label,sentence,depth_1,depth_2,depth_3
0,원인불명>조치 전 자연 회복,고객 요청사 항 안됨상담 중 자연회복됨,11,조치 전 자연 회복,NaN
1,실시간 채널>네트워크 오류,고객 요청사항 수신불인터넷 연결오류 뜸재부팅해 봄 모텔 영업용 일 중 꼭 점검원하여...,9,네트워크 오류,NaN
2,실시간 채널>수신 불량,고객 요청사항 수신불회선 단말특이사항 진단 결과 기타 추가 연락처 연후방,9,수신 불량,NaN
3,실시간 채널>네트워크 오류,수신 불 네트워크 오류 접수 보류,9,네트워크 오류,NaN
4,실시간 채널>수신 불량,고객 요청사항 수신불 연락 후 회선단말특이사항 진단 결과 기타 추가 연락처인 입,9,수신 불량,NaN


In [11]:
data_list = []
for q, label in zip(df['sentence'], df['depth_1'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [12]:
print(data_list[0])
print(data_list[3000])
print(data_list[6000])

['고객 요청사 항 안됨상담 중 자연회복됨', '11']
['티 시청 중끊김 연 후방 추티업 실패', '9']
['김주현 현장과장님 대리 접수건 고객님 댁 현장에서 대리신고 코로나 응대 고객 요청사항 티비 수신불 회선단말특이사항 진단 결과 정상신 기 타 추가 연락처', '9']


In [13]:
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [14]:
print(len(dataset_train))
print(len(dataset_test))

6139
2047


## KoBERT

#### 입력데이터 생성

In [15]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [16]:
max_len = 114
batch_size = 64
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [17]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [18]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


#### 학습 모델 만들기

In [19]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=14,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [20]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [21]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/96 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.6742289066314697 train acc 0.078125
epoch 1 train acc 0.5396418167372882


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.8842308407738095


  0%|          | 0/96 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.18329592049121857 train acc 0.953125
epoch 5 train acc 0.9319109728107344


  0%|          | 0/32 [00:00<?, ?it/s]

epoch 5 test acc 0.8895941840277778
